<a href="https://colab.research.google.com/github/YuanJ-i/qlib/blob/main/examples/workflow_by_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/microsoft/qlib/blob/main/examples/workflow_by_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#  Copyright (c) Microsoft Corporation.
#  Licensed under the MIT License.

In [2]:
import sys, site
from pathlib import Path

################################# NOTE #################################
#  Please be aware that if colab installs the latest numpy and pyqlib  #
#  in this cell, users should RESTART the runtime in order to run the  #
#  following cells successfully.                                       #
########################################################################

try:
    import qlib
except ImportError:
    # install qlib
    ! pip install --upgrade numpy
    ! pip install pyqlib
    if "google.colab" in sys.modules:
        # The Google colab environment is a little outdated. We have to downgrade the pyyaml to make it compatible with other packages
        ! pip install pyyaml==5.4.1
    # reload
    site.main()

scripts_dir = Path.cwd().parent.joinpath("scripts")
if not scripts_dir.joinpath("get_data.py").exists():
    # download get_data.py script
    scripts_dir = Path("~/tmp/qlib_code/scripts").expanduser().resolve()
    scripts_dir.mkdir(parents=True, exist_ok=True)
    import requests

    with requests.get("https://raw.githubusercontent.com/microsoft/qlib/main/scripts/get_data.py", timeout=10) as resp:
        with open(scripts_dir.joinpath("get_data.py"), "wb") as fp:
            fp.write(resp.content)

In [3]:
import qlib
import pandas as pd
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict

In [4]:
# use default data
# NOTE: need to download data from remote: python scripts/get_data.py qlib_data_cn --target_dir ~/.qlib/qlib_data/cn_data
provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
if not exists_qlib_data(provider_uri):
    print(f"Qlib data is not found in {provider_uri}")
    sys.path.append(str(scripts_dir))
    from get_data import GetData

    GetData().qlib_data(target_dir=provider_uri, region=REG_CN)
qlib.init(provider_uri=provider_uri, region=REG_CN)

Qlib data is not found in ~/.qlib/qlib_data/cn_data


2025-07-16 02:41:57.414 | WARNING  | qlib.tests.data:download:62 - The data for the example is collected from Yahoo Finance. Please be aware that the quality of the data might not be perfect. (You can refer to the original data source: https://finance.yahoo.com/lookup.)
2025-07-16 02:41:57.416 | INFO     | qlib.tests.data:download:65 - 20250716024157_qlib_data_cn_1d_latest.zip downloading......
196549632it [00:04, 41573709.51it/s]                               
2025-07-16 02:42:02.151 | WARNING  | qlib.tests.data:_unzip:124 - will delete the old qlib data directory(features, instruments, calendars, features_cache, dataset_cache): /root/.qlib/qlib_data/cn_data
2025-07-16 02:42:02.152 | INFO     | qlib.tests.data:_unzip:128 - /root/.qlib/qlib_data/cn_data/20250716024157_qlib_data_cn_1d_latest.zip unzipping......
100%|██████████| 31008/31008 [00:10<00:00, 2901.96it/s]
[769:MainThread](2025-07-16 02:42:13,344) INFO - qlib.Initialization - [config.py:420] - default_conf: client.
[769:MainTh

In [6]:
market = "csi300"
benchmark = "SH000300"

# train model

In [8]:
###################################
# train model
###################################
data_handler_config = {
    "start_time": "2008-01-01",
    "end_time": "2020-08-01",
    "fit_start_time": "2008-01-01",
    "fit_end_time": "2014-12-31",
    "instruments": market,
}

task = {
    "model": {
        "class": "LGBModel",
        "module_path": "qlib.contrib.model.gbdt",
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2008-01-01", "2014-12-31"),
                "valid": ("2015-01-01", "2016-12-31"),
                "test": ("2017-01-01", "2020-08-01"),
            },
        },
    },
}

# model initialization
model = init_instance_by_config(task["model"])

# Set joblib_backend to sequential as a workaround for the AttributeError
from qlib.config import C
C.joblib_backend = "sequential"

dataset = init_instance_by_config(task["dataset"])

# start exp to train model
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id

[769:MainThread](2025-07-16 02:49:24,317) INFO - qlib.timer - [log.py:127] - Time cost: 249.845s | Loading data Done
[769:MainThread](2025-07-16 02:49:25,382) INFO - qlib.timer - [log.py:127] - Time cost: 0.347s | DropnaLabel Done
[769:MainThread](2025-07-16 02:49:31,214) INFO - qlib.timer - [log.py:127] - Time cost: 5.830s | CSZScoreNorm Done
[769:MainThread](2025-07-16 02:49:31,219) INFO - qlib.timer - [log.py:127] - Time cost: 6.899s | fit & process data Done
[769:MainThread](2025-07-16 02:49:31,223) INFO - qlib.timer - [log.py:127] - Time cost: 256.751s | Init data Done
[769:MainThread](2025-07-16 02:49:31,261) WARNING - qlib.workflow - [expm.py:231] - No valid experiment found. Create a new experiment with name train_model.
[769:MainThread](2025-07-16 02:49:31,273) INFO - qlib.workflow - [exp.py:258] - Experiment 683977689854061117 starts running ...
[769:MainThread](2025-07-16 02:49:31,706) INFO - qlib.workflow - [recorder.py:345] - Recorder eb841561b3ff458c84aaeaab8a7f4d8b start

Training until validation scores don't improve for 50 rounds
[20]	train's l2: 0.990585	valid's l2: 0.99431
[40]	train's l2: 0.986931	valid's l2: 0.993693
[60]	train's l2: 0.984352	valid's l2: 0.99349
[80]	train's l2: 0.982319	valid's l2: 0.993382
[100]	train's l2: 0.980442	valid's l2: 0.99331
[120]	train's l2: 0.97871	valid's l2: 0.993247
[140]	train's l2: 0.976987	valid's l2: 0.993334
[160]	train's l2: 0.97536	valid's l2: 0.993338
Early stopping, best iteration is:
[122]	train's l2: 0.978519	valid's l2: 0.993238


[769:MainThread](2025-07-16 02:51:46,863) INFO - qlib.timer - [log.py:127] - Time cost: 0.237s | waiting `async_log` Done


# prediction, backtest & analysis

In [ ]:
###################################
# prediction, backtest & analysis
###################################
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs": {
            "model": model,
            "dataset": dataset,
            "topk": 50,
            "n_drop": 5,
        },
    },
    "backtest": {
        "start_time": "2017-01-01",
        "end_time": "2020-08-01",
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
with R.start(experiment_name="backtest_analysis"):
    recorder = R.get_recorder(recorder_id=rid, experiment_name="train_model")
    model = recorder.load_object("trained_model")

    # prediction
    recorder = R.get_recorder()
    ba_rid = recorder.id
    sr = SignalRecord(model, dataset, recorder)
    sr.generate()

    # backtest & analysis
    par = PortAnaRecord(recorder, port_analysis_config, "day")
    par.generate()

# analyze graphs

In [ ]:
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D

recorder = R.get_recorder(recorder_id=ba_rid, experiment_name="backtest_analysis")
print(recorder)
pred_df = recorder.load_object("pred.pkl")
report_normal_df = recorder.load_object("portfolio_analysis/report_normal_1day.pkl")
positions = recorder.load_object("portfolio_analysis/positions_normal_1day.pkl")
analysis_df = recorder.load_object("portfolio_analysis/port_analysis_1day.pkl")

## analysis position

### report

In [ ]:
analysis_position.report_graph(report_normal_df)

### risk analysis

In [ ]:
analysis_position.risk_analysis_graph(analysis_df, report_normal_df)

## analysis model

In [ ]:
label_df = dataset.prepare("test", col_set="label")
label_df.columns = ["label"]

### score IC

In [ ]:
pred_label = pd.concat([label_df, pred_df], axis=1, sort=True).reindex(label_df.index)
analysis_position.score_ic_graph(pred_label)

### model performance

In [ ]:
analysis_model.model_performance_graph(pred_label)